In [ ]:
#| hide
from launch_kit.auth import *

# launch-kit

> A FastHTML toolkit for building production-ready SaaS applications with authentication, admin panels, billing, and team management

This file will become your README and also the index of your documentation.

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install launch_kit in Development mode

```sh
# make sure launch_kit package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to launch_kit
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/LotsOfOrg/launch-kit.git
```

or from [conda][conda]

```sh
$ conda install -c LotsOfOrg launch_kit
```

or from [pypi][pypi]


```sh
$ pip install launch_kit
```


[repo]: https://github.com/LotsOfOrg/launch-kit
[docs]: https://LotsOfOrg.github.io/launch-kit/
[pypi]: https://pypi.org/project/launch-kit/
[conda]: https://anaconda.org/LotsOfOrg/launch-kit

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/LotsOfOrg/launch-kit
[docs]: https://LotsOfOrg.github.io/launch-kit/
[pypi]: https://pypi.org/project/launch-kit/
[conda]: https://anaconda.org/LotsOfOrg/launch-kit

## How to use

Launch Kit provides simple, transparent utilities for building production-ready FastHTML applications. Here's a quick example of using the authentication utilities:

In [ ]:
from launch_kit.auth import hash_password, verify_password

# Hash a password when user signs up
password = "secure_password123"
hashed = hash_password(password)
print(f"Hashed: {hashed[:20]}...")

# Verify password when user logs in
is_valid = verify_password(password, hashed)
print(f"Valid: {is_valid}")